# PyTorch 基础 : 自动求导
深度学习的算法本质上是通过反向传播求导数，而PyTorch的autograd模块则实现了此功能。在Tensor上的所有操作，autograd都能为它们自动提供微分，避免了手动计算导数的复杂过程。

***从0.4起, Variable 正式合并入Tensor, Variable 本来实现的自动微分功能，Tensor就能支持。读者还是可以使用Variable(tensor), 但是这个操作其实什么都没做。***

所以，以后的代码建议直接使用Tensor，因为官方文档中已经将Variable设置成过期模块

要想使得Tensor使用autograd功能，只需要设置tensor.requries_grad=True

In [1]:
# 首先要引入相关的包
import torch
#打印一下版本
torch.__version__

'1.1.0'

在张量创建时，通过设置 requires_grad 标识为Ture来告诉Pytorch需要对该张量进行自动求导，PyTorch会记录该张量的每一步操作历史并自动计算

In [2]:
x = torch.rand(5, 5, requires_grad=True)
x

tensor([[0.6472, 0.2715, 0.2337, 0.3190, 0.3148],
        [0.2008, 0.1859, 0.8455, 0.1581, 0.7891],
        [0.0497, 0.5365, 0.2732, 0.7512, 0.8986],
        [0.6123, 0.7268, 0.5893, 0.5975, 0.3281],
        [0.0530, 0.7999, 0.4058, 0.1836, 0.9187]], requires_grad=True)

In [3]:
y = torch.rand(5, 5, requires_grad=True)
y

tensor([[0.6604, 0.1102, 0.5281, 0.0482, 0.1964],
        [0.8315, 0.9745, 0.7399, 0.0939, 0.1111],
        [0.7787, 0.6687, 0.1736, 0.6987, 0.8521],
        [0.8834, 0.2258, 0.0585, 0.1973, 0.3289],
        [0.2477, 0.1837, 0.8077, 0.0916, 0.5026]], requires_grad=True)

我们看到 该张量的grad_fn已经被赋予了一个新的函数。下面我们来调用反向传播函数，计算其梯度

In [4]:
z=torch.sum(x+y)
z

tensor(22.6830, grad_fn=<SumBackward0>)

## 简单的自动求导

In [5]:
z.backward()

In [6]:
#看一下x和y的梯度
print(x.grad,y.grad)

tensor([[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]]) tensor([[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]])


## 复杂的自动求导

In [7]:
x = torch.rand(5, 5, requires_grad=True)
y = torch.rand(5, 5, requires_grad=True)
z= x**2+y**3
z

tensor([[0.6187, 0.6918, 0.8914, 0.2792, 0.9776],
        [0.7715, 1.2709, 1.0117, 0.8160, 0.0740],
        [0.0944, 0.7295, 0.1090, 0.2650, 0.4696],
        [0.0980, 0.1758, 0.9512, 1.4514, 0.8780],
        [0.1718, 0.1806, 1.2453, 0.6769, 0.0024]], grad_fn=<AddBackward0>)

In [8]:
#我们的返回值不是一个scalar，所以需要输入一个大小相同的张量作为参数，这里我们用ones_like函数根据x生成一个张量
z.backward(torch.ones_like(x))
print(x.grad)

tensor([[0.6264, 1.6631, 0.1671, 0.3693, 0.6895],
        [0.6006, 1.2347, 1.8366, 1.8066, 0.5070],
        [0.2486, 1.7082, 0.6548, 0.9199, 1.1607],
        [0.6261, 0.3752, 1.7224, 1.5714, 1.8739],
        [0.3230, 0.5504, 1.3369, 1.6452, 0.0853]])


我们可以使用`with torch.no_grad()`上下文管理器临时禁止对已设置requires_grad=True的张量进行自动求导。这个方法在测试集计算准确率的时候会经常用到，例如

In [9]:
with torch.no_grad():
    print((x +y*2).requires_grad)

False
